# Imports & Data Loading

In [3]:
import numpy as np
import pandas as pd

from functools import reduce

import plotly.express as px
import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')

In [4]:
df_2013 = pd.read_csv('../data/2013.csv')
df_2014 = pd.read_csv('../data/2014.csv')
df_2015 = pd.read_csv('../data/2015.csv')
df_2016 = pd.read_csv('../data/2016.csv')
df_2017 = pd.read_csv('../data/2017.csv')
df_2018 = pd.read_csv('../data/2018.csv')
df_2019 = pd.read_csv('../data/2019.csv')
df_2020 = pd.read_csv('../data/2020.csv')
df_2021 = pd.read_csv('../data/2021.csv')
df_2022 = pd.read_csv('../data/2022.csv')


dictionary_df = {
    '2013': df_2013,
    '2014': df_2014,
    '2015': df_2015,
    '2016': df_2016,
    '2017': df_2017,
    '2018': df_2018,
    '2019': df_2019,
    '2020': df_2020,
    '2021': df_2021,
    '2022': df_2022,
}

for key, df in dictionary_df.items():
    dictionary_df[key] = df[(df['Donor Agency'] == 'Bundesministerium fnr Wirtschaftliche Zusammenarbeit und Entwicklung')]
    dictionary_df[key] = dictionary_df[key][dictionary_df[key]['Sector'] != 'Administrative Costs of Donors']
    
    

# Data Processing for Globe Stacked

In [46]:
# Basic Processing of Data

def determine_value(row):
    if 2.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 1
    elif row['Climate Adaptation'] == 1.0 and row['Climate Mitigation'] == 1.0:
        return 1
    elif 1.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 0.5
    else:
        return 0

# final_dict = dictionary_df.copy()    
processed_grouped_dfs = []

    
for year, df in dictionary_df.items():
    
    # Data Cleaning
    use_df = df[['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]
    use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
    use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)
    
    # Applying analysis Rules
    
    use_df['clim_rel'] = use_df.apply(determine_value, axis=1)
    
    use_df['clim_rel_amount'] = use_df['Amount'] * use_df['clim_rel']
    use_df['clim_adapt_amount'] = use_df['Amount'] * (use_df['Climate Adaptation'] / 2)
    use_df['clim_miti_amount'] = use_df['Amount'] * (use_df['Climate Mitigation'] / 2)
    
    # Grouping by Country and calculating
    
    grouped_df = use_df.groupby(['Recipient Name']).sum()[['Amount', 'clim_rel_amount', 'clim_adapt_amount', 'clim_miti_amount']]
    
    grouped_df['clim_rel_percent'] = grouped_df['clim_rel_amount'] / grouped_df['Amount']
    
    grouped_df = grouped_df.rename(columns= {
        'Amount': f"amount_{year}",
        'clim_rel_amount': f"clim_rel_amount_{year}",
        'clim_rel_percent': f"clim_rel_percent_{year}",
        'clim_adapt_amount': f"clim_adapt_amount_{year}",
        'clim_miti_amount': f"clim_miti_amount_{year}"
    })
    
    # Merging DF
    
    processed_grouped_dfs.append(grouped_df)
    
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Recipient Name', how='outer'), processed_grouped_dfs)



## Combined

In [8]:
selected_columns = [col for col in df_merged.columns if col.startswith('amount_') or col.startswith('clim_rel_amount_')]
filtered_df = df_merged[selected_columns]

sums = filtered_df.sum()
globe_df = filtered_df.append(sums, ignore_index=True)


# globe_df = globe_df.loc[[38]]

In [9]:
globe_df

,amount_2013,clim_rel_amount_2013,amount_2014,clim_rel_amount_2014,amount_2015,clim_rel_amount_2015,amount_2016,clim_rel_amount_2016,amount_2017,clim_rel_amount_2017,amount_2018,clim_rel_amount_2018,amount_2019,clim_rel_amount_2019,amount_2020,clim_rel_amount_2020,amount_2021,clim_rel_amount_2021,amount_2022,clim_rel_amount_2022
0,229.878097,19.054332,286.807860,31.714736,197.429500,33.457037,346.234112,38.308328,311.670340,42.762633,255.815158,41.988156,241.099156,40.556326,196.114075,34.977410,352.840862,51.961490,201.939382,16.806849
1,132.706037,34.375215,129.848523,17.713145,182.614727,79.600243,129.000894,21.123321,240.258759,109.944048,262.731752,63.757281,555.037032,147.663336,686.808006,180.211271,643.229422,136.808525,627.644704,195.336748
2,19.317268,5.281618,16.371641,5.216575,19.155382,6.609535,18.968980,6.652384,21.017326,7.728618,55.514401,44.503031,19.355900,5.719183,30.080577,7.164332,29.366152,10.070879,25.328362,9.591473
3,6.080655,2.350579,5.218615,4.723721,6.014444,5.011871,4.684949,3.522966,5.347355,4.006726,6.986998,5.201791,9.983005,6.724355,11.714897,5.986887,25.777176,7.808470,16.273651,8.764504
4,61.039902,16.707576,73.783053,35.047207,48.050705,6.148801,77.236341,29.640753,95.230412,58.889436,59.652114,26.355707,103.276123,56.378290,146.314609,103.204088,82.869791,34.533597,107.665968,67.062139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.232934,3.934650,27.979557,9.016008,31.071347,2.476874,31.393523,3.293854
139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.435574,4.101530,5.156201,2.822914,7.093164,3.505433,7.352916,3.160977
140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.828161,7.896584,34.419681,9.418930,29.927608,9.239338,37.365181,18.475603
141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.119008,0.000000,15.884541,2.771182


In [43]:
years = range(2013, 2023)

for year in years:
    amount_col = f'amount_{year}'
    clim_rel_amount_col = f'clim_rel_amount_{year}'
    non_clim_col = f'non_clim_amount_{year}'
    globe_df[non_clim_col] = globe_df[amount_col] - globe_df[clim_rel_amount_col]

# Step 2: Melt the dataframe for plotting
# Including both climatic and non-climatic amounts
melted_df = globe_df.melt(value_vars=[f'clim_rel_amount_{year}' for year in years] + 
                             [f'non_clim_amount_{year}' for year in years],
                             var_name='Type_Year', value_name='Amount')



# Split 'Type_Year' into separate 'Year' and 'Type' columns
melted_df['Year'] = melted_df['Type_Year'].apply(lambda x: x.split('_')[-1])
melted_df['Finanzierungstyp'] = melted_df['Type_Year'].apply(lambda x: 'Climate Finance' if 'clim_rel_amount' in x else 'Other Funds')

melted_df['Amount'] = melted_df['Amount'] * 1_000_000

replacements = {
    'Climate Finance': 'Klimafinanzierung',
    'Other Funds': 'Andere ODA'
}
melted_df['Finanzierungstyp'] = melted_df['Finanzierungstyp'].replace(replacements)

melted_df.to_csv("../upload_data/globe_df.csv")

In [44]:
melted_df

,Type_Year,Amount,Year,Finanzierungstyp
0,clim_rel_amount_2013,9.921158e+08,2013,Klimafinanzierung
1,clim_rel_amount_2014,1.197406e+09,2014,Klimafinanzierung
2,clim_rel_amount_2015,1.152733e+09,2015,Klimafinanzierung
3,clim_rel_amount_2016,1.660725e+09,2016,Klimafinanzierung
4,clim_rel_amount_2017,1.715085e+09,2017,Klimafinanzierung
5,clim_rel_amount_2018,1.904175e+09,2018,Klimafinanzierung
6,clim_rel_amount_2019,2.310213e+09,2019,Klimafinanzierung
7,clim_rel_amount_2020,2.563919e+09,2020,Klimafinanzierung
8,clim_rel_amount_2021,2.972043e+09,2021,Klimafinanzierung
9,clim_rel_amount_2022,3.333997e+09,2022,Klimafinanzierung


## Split

In [50]:
selected_columns = [col for col in df_merged.columns if col.startswith('amount_') or col.startswith('clim_rel_amount_')\
                   or col.startswith('clim_adapt_') or col.startswith('clim_miti_')]
filtered_df = df_merged[selected_columns]

sums = filtered_df.sum()
globe_df = filtered_df.append(sums, ignore_index=True)
globe_df = globe_df.loc[[142]]


years = range(2013, 2023)

for year in years:
    amount_col = f'amount_{year}'
    clim_adapt_col = f'clim_adapt_amount_{year}'
    clim_miti_col = f'clim_miti_amount_{year}'
    non_clim_col = f'non_clim_amount_{year}'
    globe_df[non_clim_col] = globe_df[amount_col] - (globe_df[clim_adapt_col] + globe_df[clim_miti_col])


# Step 2: Melt the dataframe for plotting
# Including both climatic and non-climatic amounts
melted_df = globe_df.melt(value_vars=[f'clim_adapt_amount_{year}' for year in years] +
                          [f'clim_miti_amount_{year}' for year in years] +
                             [f'non_clim_amount_{year}' for year in years],
                             var_name='Type_Year', value_name='Amount')


# Split 'Type_Year' into separate 'Year' and 'Type' columns
melted_df['Year'] = melted_df['Type_Year'].apply(lambda x: x.split('_')[-1])
melted_df['Finanzierungstyp'] = melted_df['Type_Year'].apply(
    lambda x: 'Klimaanpassung Finanzierung' if 'clim_adapt_amount' in x 
    else ('Klimaschutz Finanzierung' if 'clim_miti_amount' in x 
    else 'Andere ODA')
)
melted_df['Amount'] = melted_df['Amount'] * 1_000_000




In [52]:
melted_df.to_csv("../upload_data/split_globe.csv")

read_df = pd.read_csv('../upload_data/split_globe.csv')


In [59]:
split_df = pd.read_csv('../upload_data/split_globe.csv')

total_per_year = split_df.groupby('Year')['Amount'].transform('sum')


split_df['Percentage'] = (split_df['Amount'] / total_per_year) * 100




In [111]:
fig = px.bar(split_df, x='Year', y='Percentage', color='Finanzierungstyp',
            title='Globale Finanzierungssummen (Prozentual)',
            labels={'Percentage': 'Prozentsatz der Finanzierung', 'Year': 'Jahr'},
            category_orders={'Finanzierungstyp': ['Andere ODA', 'Klimaschutz Finanzierung', 'Klimaanpassung Finanzierung']},
            color_discrete_map={'Andere ODA': 'orange', 'Klimaschutz Finanzierung': 'green', 'Klimaanpassung Finanzierung': 'blue'}
            )

# Centering the title and adjusting layout
fig.update_layout(title_x=0.5)

# Show the plot
fig.show()

In [116]:
split_df = pd.read_csv('../upload_data/split_globe.csv')


fig = px.bar(split_df, x='Year', y='Amount', color='Finanzierungstyp',
            title='Globale Finanzierungssummen',
            labels={'Amount': 'Finanzierungssumme ($)', 'Year': 'Jahr'},
            category_orders={'Finanzierungstyp': ['Andere ODA','Klimaschutz Finanzierung', 'Klimaanpassung Finanzierung']},
            color_discrete_map={'Andere ODA': 'orange', 'Klimaschutz Finanzierung': 'green', 'Klimaanpassung Finanzierung': 'blue'}
            )# This ensures consistent color ordering

fig.update_layout(title_x=0.5)
tick_values = [2e9, 4e9, 6e9, 8e9, 10e9, 12e9]
tick_labels = ['2 M', '4 M', '6 M', '8 M', '10 M', '12 M']

# Update the y-axis to display values in billions ('M' for Milliarden)
fig.update_yaxes(tickprefix="", ticksuffix="",
                 tickvals=tick_values,
                 ticktext=tick_labels)
fig.show()

# Waterfall

In [10]:
globe_df = globe_df.loc[[142]]

In [11]:
globe_df

,amount_2013,clim_rel_amount_2013,amount_2014,clim_rel_amount_2014,amount_2015,clim_rel_amount_2015,amount_2016,clim_rel_amount_2016,amount_2017,clim_rel_amount_2017,amount_2018,clim_rel_amount_2018,amount_2019,clim_rel_amount_2019,amount_2020,clim_rel_amount_2020,amount_2021,clim_rel_amount_2021,amount_2022,clim_rel_amount_2022
142,5115.784928,992.115751,6323.119826,1197.406245,5316.503434,1152.733316,6897.008022,1660.725242,7204.68697,1715.085188,8012.691583,1904.175131,8632.582927,2310.213481,10294.707282,2563.918782,10593.415129,2972.043199,11452.254162,3333.997495


In [12]:
years = [col.split('_')[-1] for col in globe_df.columns if col.startswith('amount_')]

for year in years:
    amount_col = f'amount_{year}'
    clim_rel_amount_col = f'clim_rel_amount_{year}'
    clim_rel_percent_col = f'clim_rel_percent_{year}'
    
    # Calculate the clim_rel_percent column for the year
    # Note: It's a good practice to handle division by zero or check if 'amount_' is not zero to avoid ZeroDivisionError.
    globe_df[clim_rel_percent_col] = (globe_df[clim_rel_amount_col] / globe_df[amount_col]) * 100
    
    

selected_columns = [col for col in globe_df.columns if col.startswith('clim_rel_percent_')]
filtered_df = globe_df[selected_columns]


# Melt the DataFrame to get it in the right shape for plotting
melted_df = filtered_df.reset_index().melt(id_vars=['index'], var_name='Year', value_name='Percentage')

# Clean the 'Year' column to keep only the year part
melted_df['Year'] = melted_df['Year'].str.replace('clim_rel_percent_', '')

melted_df = melted_df.drop(columns = 'index')

melted_df['Change'] = melted_df['Percentage'].diff()

melted_df

,Year,Percentage,Change
0,2013,19.393226,NaN
1,2014,18.936953,-0.456273
2,2015,21.682170,2.745217
3,2016,24.078923,2.396753
4,2017,23.805131,-0.273792
5,2018,23.764488,-0.040643
6,2019,26.761556,2.997068
7,2020,24.905213,-1.856343
8,2021,28.055572,3.150359
9,2022,29.112151,1.056579


In [13]:
melted_df.to_csv('../upload_data/globe_waterfall.csv')

# globe_df_to_show

In [18]:
globe_df

,amount_2013,clim_rel_amount_2013,amount_2014,clim_rel_amount_2014,amount_2015,clim_rel_amount_2015,amount_2016,clim_rel_amount_2016,amount_2017,clim_rel_amount_2017,...,clim_rel_percent_2013,clim_rel_percent_2014,clim_rel_percent_2015,clim_rel_percent_2016,clim_rel_percent_2017,clim_rel_percent_2018,clim_rel_percent_2019,clim_rel_percent_2020,clim_rel_percent_2021,clim_rel_percent_2022
142,5115.784928,992.115751,6323.119826,1197.406245,5316.503434,1152.733316,6897.008022,1660.725242,7204.68697,1715.085188,...,19.393226,18.936953,21.68217,24.078923,23.805131,23.764488,26.761556,24.905213,28.055572,29.112151


In [90]:
df_melted = pd.melt(globe_df, var_name='variable', value_name='value')

# Extract year and type
df_melted['year'] = df_melted['variable'].str.extract(r'(\d+)$')[0].astype(int)
df_melted['type'] = df_melted['variable'].str.extract(r'^(.*?)_\d+$')[0]

# Pivot table to restructure into the desired format
df_final = df_melted.pivot_table(index='year', columns='type', values='value', aggfunc='first').reset_index()

# Rename columns as needed
df_final.columns.name = None  # Remove the pivot table multi-level indexing name

df_final = df_final.rename(columns= {'amount': 'Gesamtsumme (Mrd. $)', 
                          'clim_rel_amount': 'davon Klimafinanzierung',
                          'clim_adapt_amount': 'davon Klimaanpassungsfinanzierung',
                                     'clim_miti_amount': 'davon Klimaschutzfinanzierung',
                                     'year':'Jahr'
                         }).set_index('Jahr')[['Gesamtsumme (Mrd. $)', 'davon Klimafinanzierung','davon Klimaanpassungsfinanzierung', 'davon Klimaschutzfinanzierung']]

In [91]:
df_final

,Gesamtsumme (Mrd. $),davon Klimafinanzierung,davon Klimaanpassungsfinanzierung,davon Klimaschutzfinanzierung
Jahr,,,,
2013,5115.784928,992.115751,430.780688,561.335063
2014,6323.119826,1197.406245,599.863973,597.542272
2015,5316.503434,1152.733316,661.362943,491.370374
2016,6897.008022,1660.725242,893.345489,767.379753
2017,7204.686970,1715.085188,865.653914,849.431275
2018,8012.691583,1904.175131,1048.374900,855.800231
2019,8632.582927,2310.213481,1242.999296,1067.214184
2020,10294.707282,2563.918782,1509.883531,1054.035251
2021,10593.415129,2972.043199,1596.187821,1376.035353


In [92]:
df_final.to_csv('../upload_data/globe_df_to_show.csv')

In [93]:
pd.read_csv('../upload_data/globe_df_to_show.csv')

,Jahr,Gesamtsumme (Mrd. $),davon Klimafinanzierung,davon Klimaanpassungsfinanzierung,davon Klimaschutzfinanzierung
0,2013,5115.784928,992.115751,430.780688,561.335063
1,2014,6323.119826,1197.406245,599.863973,597.542272
2,2015,5316.503434,1152.733316,661.362943,491.370374
3,2016,6897.008022,1660.725242,893.345489,767.379753
4,2017,7204.686970,1715.085188,865.653914,849.431275
5,2018,8012.691583,1904.175131,1048.374900,855.800231
6,2019,8632.582927,2310.213481,1242.999296,1067.214185
7,2020,10294.707282,2563.918782,1509.883531,1054.035251
8,2021,10593.415129,2972.043199,1596.187821,1376.035353
9,2022,11452.254162,3333.997495,1693.406647,1640.856550


# all_country_df.csv

In [157]:
# Basic Processing of Data

def determine_value(row):
    if 2.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 1
    elif 1.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 0.5
    else:
        return 0

# final_dict = dictionary_df.copy()    
processed_grouped_dfs = []

    
for year, df in dictionary_df.items():
    
    # Data Cleaning
    use_df = df[['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]
    use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
    use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)
    
    # Applying analysis Rules
    
    use_df['clim_rel'] = use_df.apply(determine_value, axis=1)
    
    use_df['clim_rel_amount'] = use_df['Amount'] * use_df['clim_rel']
    
    # Grouping by Country and calculating
    
    grouped_df = use_df.groupby(['Recipient Name']).sum()[['Amount', 'clim_rel_amount']]
    
    grouped_df['clim_rel_percent'] = grouped_df['clim_rel_amount'] / grouped_df['Amount']
    
    grouped_df = grouped_df.rename(columns= {
        'Amount': f"amount_{year}",
        'clim_rel_amount': f"clim_rel_amount_{year}",
        'clim_rel_percent': f"clim_rel_percent_{year}"
    })
    
    # Merging DF
    
    processed_grouped_dfs.append(grouped_df)
    
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Recipient Name', how='outer'), processed_grouped_dfs)



# df_merged.index = df_merged.index.rename('Empfängerland')

# years = range(2013, 2023)  # Adjust end year as necessary to include all years you have

# for year in years:
#     rename_dict.update({
#         f'amount_{year}': f'Summe {year}',
#         f'clim_rel_amount_{year}': f'Klimarelevante Summe {year}',
#         f'clim_rel_percent_{year}': f'Klimarelevanter Prozentsatz {year}'
#     })

# df_merged.rename(columns=rename_dict, inplace=True)

df_merged.to_csv("../upload_data/all_country_df.csv")


# All country df to download

In [159]:
# Basic Processing of Data

def determine_value(row):
    if 2.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 1
    elif 1.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 0.5
    else:
        return 0

# final_dict = dictionary_df.copy()    
processed_grouped_dfs = []

    
for year, df in dictionary_df.items():
    
    # Data Cleaning
    use_df = df[['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]
    use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
    use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)
    
    # Applying analysis Rules
    
    use_df['clim_rel'] = use_df.apply(determine_value, axis=1)
    
    use_df['clim_rel_amount'] = use_df['Amount'] * use_df['clim_rel']
    
    # Grouping by Country and calculating
    
    grouped_df = use_df.groupby(['Recipient Name']).sum()[['Amount', 'clim_rel_amount']]
    
    grouped_df['clim_rel_percent'] = grouped_df['clim_rel_amount'] / grouped_df['Amount']
    
    grouped_df = grouped_df.rename(columns= {
        'Amount': f"amount_{year}",
        'clim_rel_amount': f"clim_rel_amount_{year}",
        'clim_rel_percent': f"clim_rel_percent_{year}"
    })
    
    # Merging DF
    
    processed_grouped_dfs.append(grouped_df)
    
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Recipient Name', how='outer'), processed_grouped_dfs)



df_merged.index = df_merged.index.rename('Empfängerland')

years = range(2013, 2023)  # Adjust end year as necessary to include all years you have

for year in years:
    rename_dict.update({
        f'amount_{year}': f'Summe {year}',
        f'clim_rel_amount_{year}': f'Klimarelevante Summe {year}',
        f'clim_rel_percent_{year}': f'Klimarelevanter Prozentsatz {year}'
    })

df_merged.rename(columns=rename_dict, inplace=True)

df_merged.to_csv("../upload_data/all_country_df_download.csv")


In [160]:
df_merged

,Summe 2013,Klimarelevante Summe 2013,Klimarelevanter Prozentsatz 2013,Summe 2014,Klimarelevante Summe 2014,Klimarelevanter Prozentsatz 2014,Summe 2015,Klimarelevante Summe 2015,Klimarelevanter Prozentsatz 2015,Summe 2016,...,Klimarelevanter Prozentsatz 2019,Summe 2020,Klimarelevante Summe 2020,Klimarelevanter Prozentsatz 2020,Summe 2021,Klimarelevante Summe 2021,Klimarelevanter Prozentsatz 2021,Summe 2022,Klimarelevante Summe 2022,Klimarelevanter Prozentsatz 2022
Empfängerland,,,,,,,,,,,,,,,,,,,,,
Afghanistan,229.878097,18.110155,0.078782,286.807860,30.496058,0.106329,197.429500,30.311894,0.153533,346.234112,...,0.165281,196.114075,33.459576,0.170613,352.840862,47.657621,0.135068,201.939382,12.035070,0.059597
"Africa, regional",132.706037,34.237272,0.257993,129.848523,15.707783,0.120970,182.614727,75.974172,0.416035,129.000894,...,0.234096,686.808006,161.181838,0.234683,643.229422,131.993625,0.205205,627.644704,138.891343,0.221290
Albania,19.317268,5.281618,0.273414,16.371641,4.651060,0.284092,19.155382,6.362595,0.332157,18.968980,...,0.271722,30.080577,5.804841,0.192976,29.366152,7.881105,0.268374,25.328362,8.592429,0.339241
Algeria,6.080655,1.175290,0.193283,5.218615,2.543035,0.487301,6.014444,3.463474,0.575859,4.684949,...,0.455295,11.714897,4.716805,0.402633,25.777176,6.526832,0.253202,16.273651,7.171532,0.440684
"America, regional",61.039902,12.957501,0.212279,73.783053,20.492112,0.277735,48.050705,5.792946,0.120559,77.236341,...,0.391508,146.314609,54.032500,0.369290,82.869791,29.554400,0.356637,107.665968,50.903984,0.472795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Central America, regional",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.723529,6.512341,3.309877,0.508247,7.760249,3.882104,0.500255,9.319793,4.376188,0.469558
"Eastern Africa, regional",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.165010,27.979557,6.321787,0.225943,31.071347,2.476874,0.079716,31.393523,3.239855,0.103201
"Middle Africa, regional",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.470844,5.156201,1.411457,0.273740,7.093164,1.752717,0.247099,7.352916,1.580488,0.214947


# All country Split

In [123]:
# Basic Processing of Data

def determine_value(row):
    if 2.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 1
    elif row['Climate Adaptation'] == 1.0 and row['Climate Mitigation'] == 1.0:
        return 1
    elif 1.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 0.5
    else:
        return 0

# final_dict = dictionary_df.copy()    
processed_grouped_dfs = []

    
for year, df in dictionary_df.items():
    
    # Data Cleaning
    use_df = df[['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]
    use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
    use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)
    
    # Applying analysis Rules
    
    use_df['clim_rel'] = use_df.apply(determine_value, axis=1)
    
    use_df['clim_rel_amount'] = use_df['Amount'] * use_df['clim_rel']
    use_df['clim_adapt_amount'] = use_df['Amount'] * (use_df['Climate Adaptation'] / 2)
    use_df['clim_miti_amount'] = use_df['Amount'] * (use_df['Climate Mitigation'] / 2)
    
    # Grouping by Country and calculating
    
    grouped_df = use_df.groupby(['Recipient Name']).sum()[['Amount', 'clim_rel_amount', 'clim_adapt_amount', 'clim_miti_amount']]
    
    grouped_df['clim_rel_percent'] = grouped_df['clim_rel_amount'] / grouped_df['Amount']
    
    grouped_df = grouped_df.rename(columns= {
        'Amount': f"amount_{year}",
        'clim_rel_amount': f"clim_rel_amount_{year}",
        'clim_rel_percent': f"clim_rel_percent_{year}",
        'clim_adapt_amount': f"clim_adapt_amount_{year}",
        'clim_miti_amount': f"clim_miti_amount_{year}"
    })
    
    # Merging DF
    
    processed_grouped_dfs.append(grouped_df)
    
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Recipient Name', how='outer'), processed_grouped_dfs)



In [137]:
selected_columns = [col for col in df_merged.columns if col.startswith('amount_') or col.startswith('clim_rel_amount_')\
                   or col.startswith('clim_adapt_') or col.startswith('clim_miti_')]
filtered_df = df_merged[selected_columns]

# sums = filtered_df.sum()
# globe_df = filtered_df.append(sums, ignore_index=True)
# globe_df = globe_df.loc[[142]]


years = range(2013, 2023)

for year in years:
    amount_col = f'amount_{year}'
    clim_adapt_col = f'clim_adapt_amount_{year}'
    clim_miti_col = f'clim_miti_amount_{year}'
    non_clim_col = f'non_clim_amount_{year}'
    filtered_df[non_clim_col] = filtered_df[amount_col] - (filtered_df[clim_adapt_col] + filtered_df[clim_miti_col])


# Step 2: Melt the dataframe for plotting
# Including both climatic and non-climatic amounts
filtered_df = filtered_df.reset_index()

melted_df = filtered_df.melt(id_vars=['Recipient Name'],value_vars=[f'clim_adapt_amount_{year}' for year in years] +
                          [f'clim_miti_amount_{year}' for year in years] +
                             [f'non_clim_amount_{year}' for year in years],
                             var_name='Type_Year', value_name='Amount')


# Split 'Type_Year' into separate 'Year' and 'Type' columns
melted_df['Year'] = melted_df['Type_Year'].apply(lambda x: x.split('_')[-1])
melted_df['Finanzierungstyp'] = melted_df['Type_Year'].apply(
    lambda x: 'Klimaanpassung Finanzierung' if 'clim_adapt_amount' in x 
    else ('Klimaschutz Finanzierung' if 'clim_miti_amount' in x 
    else 'Andere ODA')
)
melted_df['Amount'] = melted_df['Amount'] * 1_000_000




In [139]:
melted_df.to_csv("../upload_data/all_country_split.csv")


In [142]:
country = 'India'

In [145]:
all_country_split_df = pd.read_csv('../upload_data/all_country_split.csv')

selected_country_split_df = all_country_split_df[all_country_split_df['Recipient Name'] == country]

fig = px.bar(selected_country_split_df, x='Year', y='Amount', color='Finanzierungstyp',
            title='Globale Finanzierungssummen',
            labels={'Amount': 'Finanzierungssumme ($)', 'Year': 'Jahr'},
            category_orders={'Finanzierungstyp': ['Andere ODA','Klimaschutz Finanzierung', 'Klimaanpassung Finanzierung']},
            color_discrete_map={'Andere ODA': 'orange', 'Klimaschutz Finanzierung': 'green', 'Klimaanpassung Finanzierung': 'blue'}
            )# This ensures consistent color ordering

fig.show()

In [118]:
df_merged

,amount_2013,clim_rel_amount_2013,clim_rel_percent_2013,amount_2014,clim_rel_amount_2014,clim_rel_percent_2014,amount_2015,clim_rel_amount_2015,clim_rel_percent_2015,amount_2016,...,clim_rel_percent_2019,amount_2020,clim_rel_amount_2020,clim_rel_percent_2020,amount_2021,clim_rel_amount_2021,clim_rel_percent_2021,amount_2022,clim_rel_amount_2022,clim_rel_percent_2022
Recipient Name,,,,,,,,,,,,,,,,,,,,,
Afghanistan,229.878097,18.110155,0.078782,286.807860,30.496058,0.106329,197.429500,30.311894,0.153533,346.234112,...,0.165281,196.114075,33.459576,0.170613,352.840862,47.657621,0.135068,201.939382,12.035070,0.059597
"Africa, regional",132.706037,34.237272,0.257993,129.848523,15.707783,0.120970,182.614727,75.974172,0.416035,129.000894,...,0.234096,686.808006,161.181838,0.234683,643.229422,131.993625,0.205205,627.644704,138.891343,0.221290
Albania,19.317268,5.281618,0.273414,16.371641,4.651060,0.284092,19.155382,6.362595,0.332157,18.968980,...,0.271722,30.080577,5.804841,0.192976,29.366152,7.881105,0.268374,25.328362,8.592429,0.339241
Algeria,6.080655,1.175290,0.193283,5.218615,2.543035,0.487301,6.014444,3.463474,0.575859,4.684949,...,0.455295,11.714897,4.716805,0.402633,25.777176,6.526832,0.253202,16.273651,7.171532,0.440684
"America, regional",61.039902,12.957501,0.212279,73.783053,20.492112,0.277735,48.050705,5.792946,0.120559,77.236341,...,0.391508,146.314609,54.032500,0.369290,82.869791,29.554400,0.356637,107.665968,50.903984,0.472795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Central America, regional",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.723529,6.512341,3.309877,0.508247,7.760249,3.882104,0.500255,9.319793,4.376188,0.469558
"Eastern Africa, regional",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.165010,27.979557,6.321787,0.225943,31.071347,2.476874,0.079716,31.393523,3.239855,0.103201
"Middle Africa, regional",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.470844,5.156201,1.411457,0.273740,7.093164,1.752717,0.247099,7.352916,1.580488,0.214947


# country_specific_df

In [39]:
processed_grouped_dfs = []

def determine_value(row):
    if 2.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 1
    elif 1.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 0.5
    else:
        return 0

for key in dictionary_df.keys():
        
    use_df = dictionary_df[key][['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]

    use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
    use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)

    use_df['clim_rel'] = use_df.apply(determine_value, axis=1)
    use_df['clim_rel_amount'] = use_df['Amount'] * use_df['clim_rel']
    
    grouped_df = use_df.groupby(['Recipient Name']).sum()[['Amount', 'clim_rel_amount']].reset_index()
    
    grouped_df['clim_rel_percent'] = grouped_df['clim_rel_amount'] / grouped_df['Amount']
    
    grouped_processed_df = grouped_df.rename(columns={'clim_rel_percent': key})

    processed_grouped_dfs.append(grouped_processed_df[['Recipient Name',key]])

    
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Recipient Name', how='outer'), processed_grouped_dfs)

df_long = df_merged.melt(id_vars=["Recipient Name"], 
                  var_name="Year", 
                  value_name="Value")


df_long.to_csv('../upload_data/country_specific_df.csv')



# Data Processing For Sector Breakdown

In [161]:
def determine_value(row):
    if 2.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 1
    elif row['Climate Adaptation'] == 1.0 and row['Climate Mitigation'] == 1.0:
        return 1
    elif 1.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 0.5
    else:
        return 0

# final_dict = dictionary_df.copy()    
processed_grouped_dfs = []

    
for year, df in dictionary_df.items():
    
    # Data Cleaning
    use_df = df[['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation', 'Sector']]
    use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
    use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)
    
    # Applying analysis Rules
    
    use_df['clim_rel'] = use_df.apply(determine_value, axis=1)
    
    use_df['clim_rel_amount'] = use_df['Amount'] * use_df['clim_rel']
    use_df['clim_adapt_amount'] = use_df['Amount'] * (use_df['Climate Adaptation'] / 2)
    use_df['clim_miti_amount'] = use_df['Amount'] * (use_df['Climate Mitigation'] / 2)
    
    
    # Grouping by Country and calculating
    
    grouped_df = use_df.groupby('Sector').sum()[['Amount','clim_rel_amount', 'clim_adapt_amount', 'clim_miti_amount']]
    
    grouped_df['non_miti_adapt'] =grouped_df['Amount'] - (grouped_df['clim_miti_amount'] + grouped_df['clim_adapt_amount'])
    grouped_df['non_clim'] =grouped_df['Amount'] - grouped_df['clim_rel_amount']
    
        
    grouped_df = grouped_df.rename(columns= {
        'Amount': f"amount_{year}",
        'non_miti_adapt': f"non_miti_adapt_{year}",
        'clim_rel_amount': f"clim_rel_amount_{year}",
        'non_clim': f"non_clim_{year}",
        'clim_adapt_amount': f"clim_adapt_amount_{year}",
        'clim_miti_amount': f"clim_miti_amount_{year}"
    })
    
    # Merging DF
    
    processed_grouped_dfs.append(grouped_df)
    



In [177]:
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Sector', how='outer'), processed_grouped_dfs)


In [181]:
df_merged = df_merged*1000000

In [182]:
df_merged.to_csv('../upload_data/sector_analysis.csv')

df_merged = pd.read_csv('../upload_data/sector_analysis.csv')

## Stacked Area

In [209]:
df_merged.head()

,Sector,amount_2013,clim_rel_amount_2013,clim_adapt_amount_2013,clim_miti_amount_2013,non_miti_adapt_2013,non_clim_2013,amount_2014,clim_rel_amount_2014,clim_adapt_amount_2014,...,clim_adapt_amount_2021,clim_miti_amount_2021,non_miti_adapt_2021,non_clim_2021,amount_2022,clim_rel_amount_2022,clim_adapt_amount_2022,clim_miti_amount_2022,non_miti_adapt_2022,non_clim_2022
0,"I.1.a. Education, Level Unspecified",87245749.0,6281512.0,3838428.0,2443084.0,80964237.0,80964237.0,73771448.0,5642177.0,4251977.5,...,870306.0,666370.5,250891593.5,250891593.5,265014798.0,2829080.0,1364922.0,1464158.0,262185718.0,262185718.0
1,I.1.b. Basic Education,153930790.0,185663.5,139590.5,46073.0,153745126.5,153745126.5,169047826.0,158667.0,112319.5,...,380428.0,181201.0,269793665.0,269793665.0,237274430.0,830313.5,566782.0,263531.5,236444116.5,236444116.5
2,I.1.c. Secondary Education,110570602.0,4461417.5,3382913.0,1078504.5,106109184.5,106109184.5,123680119.0,5417927.5,3629494.5,...,18651157.0,6372143.5,313863098.5,313863098.5,336204882.0,24097309.0,17452875.5,6644433.5,312107573.0,312107573.0
3,I.1.d. Post-Secondary Education,104939416.0,10369314.5,3066040.0,7303274.5,94570101.5,94570101.5,91545807.0,3192433.0,2932055.5,...,2036140.5,2501237.5,95873669.0,95873669.0,107771939.0,5137190.5,1959193.5,3177997.0,102634748.5,102634748.5
4,"I.2.a. Health, General",95765407.0,24583.5,0.0,24583.5,95740823.5,95740823.5,86811676.0,3661154.0,3661154.0,...,843957.0,654806.5,164711020.5,164711020.5,174278158.0,641030.5,493282.0,147748.5,173637127.5,173637127.5


In [227]:
a_year_start = [2013, 2014, 2015]
b_year_end = [2020,2021, 2022]
c_category = ["Gesamt ODA", "Klimarelevant", "Klimaschutz", "Klimaanpassung"]
d_top_n_sectors = [3,5,10,15]
e_abs_or_perc = ["Absolute Werte", "Anteilig"]

In [303]:
def stacked_area_chart(full_sector_df, year_start, year_end, category, top_n_sectors, abs_or_perc):
    
    # Select Category
    
    translate_dict = {
        "Gesamt ODA": "amount",
        "Klimarelevant": "clim_rel",
        "Klimaschutz": "clim_miti",
        "Klimaanpassung": "clim_adapt"
    }
    
    selected_cols = [col for col in full_sector_df.columns if col.startswith(translate_dict[category])]
    selected_cols.append('Sector')
    filtered_col_df = full_sector_df[selected_cols]
    
    # Select Top_N
    
    sector_avg = filtered_col_df.melt(id_vars=['Sector'], var_name='Year', value_name='Amount').groupby('Sector').mean()
    top_sectors = sector_avg['Amount'].nlargest(top_n_sectors).index.tolist()
    filtered_col_df['Grouped Sector'] = filtered_col_df['Sector'].apply(lambda x: x if x in top_sectors else 'Andere Sektoren')
    grouped_df = filtered_col_df.groupby('Grouped Sector').sum().reset_index()
    
    # Prepare for Plotting
    
    long_df = grouped_df.melt(id_vars=['Grouped Sector'], var_name='Year', value_name='Amount')
    long_df['Year'] = long_df['Year'].str.extract('(\d+)')
    long_df['Year'] = pd.to_numeric(long_df['Year'])
    
    # Year Select
    
    long_df = long_df[long_df['Year'].between(year_start,year_end)]
    
    # Option for Percentage Plot
    
    if abs_or_perc == 'percent':
        
        # Calculate Percent
        
        total_per_year = long_df.groupby('Year')['Amount'].sum().reset_index(name='Total')
        long_df = pd.merge(long_df, total_per_year, on='Year')
        long_df['Percentage'] = (long_df['Amount'] / long_df['Total']) * 100
        
        # Create a filled area plot

        fig = px.area(long_df, x='Year', y='Percentage', color='Grouped Sector',
                      labels={'Percentage': 'Percentage of Total'},
                      title='Stacked Area Plot of Grouped Sector as Percentage of Total per Year')

        fig.update_layout(
            paper_bgcolor='white',
            plot_bgcolor='white',
            xaxis=dict(showgrid=False),
            yaxis=dict(showgrid=False, ticksuffix="%")  # Add a percentage sign to y-axis ticks
        )

    
        return fig

    # Create a filled area plot
    
    fig = px.area(long_df, x='Year', y='Amount', color='Grouped Sector')
    fig.update_layout(
        paper_bgcolor='white',
        plot_bgcolor='white',
        )
    
    return fig
    
    
    
    

In [311]:
sector_anal_df = pd.read_csv('../upload_data/sector_analysis.csv')

def stacked_area_chart(full_sector_df, year_start, year_end, category, top_n_sectors, abs_or_perc):
    
    # Select Category
    
    translate_dict = {
        "Gesamt ODA": "amount",
        "Klimarelevant": "clim_rel",
        "Klimaschutz": "clim_miti",
        "Klimaanpassung": "clim_adapt"
    }
    
    selected_cols = [col for col in full_sector_df.columns if col.startswith(translate_dict[category])]
    selected_cols.append('Sector')
    filtered_col_df = full_sector_df[selected_cols]
        
    # Select Top_N
    
    sector_avg = filtered_col_df.melt(id_vars=['Sector'], var_name='Year', value_name='Amount').groupby('Sector').mean()
    top_sectors = sector_avg['Amount'].nlargest(top_n_sectors).index.tolist()
    filtered_col_df['Grouped Sector'] = filtered_col_df['Sector'].apply(lambda x: x if x in top_sectors else 'Andere Sektoren')
    grouped_df = filtered_col_df.groupby('Grouped Sector').sum().reset_index()

    
    # Prepare for Plotting
    
    long_df = grouped_df.melt(id_vars=['Grouped Sector'], var_name='Year', value_name='Amount')
    long_df['Year'] = long_df['Year'].str.extract('(\d+)')
    long_df['Year'] = pd.to_numeric(long_df['Year'])
    
    return long_df

    
    # Year Select
    
    long_df = long_df[long_df['Year'].between(year_start,year_end)]
    
    # Option for Percentage Plot
    
    if abs_or_perc == 'Anteilig':
        
        # Calculate Percent
        
        total_per_year = long_df.groupby('Year')['Amount'].sum().reset_index(name='Total')
        long_df = pd.merge(long_df, total_per_year, on='Year')
        long_df['Percentage'] = (long_df['Amount'] / long_df['Total']) * 100
        
        # Create a filled area plot

        fig = px.area(long_df, x='Year', y='Percentage', color='Grouped Sector',
                      labels={'Percentage': 'Percentage of Total'},
                      title='Stacked Area Plot of Grouped Sector as Percentage of Total per Year')

        fig.update_layout(
            paper_bgcolor='white',
            plot_bgcolor='white',
            xaxis=dict(showgrid=False),
            yaxis=dict(showgrid=False, ticksuffix="%")  # Add a percentage sign to y-axis ticks
        )

    
        return fig

    # Create a filled area plot
    
    fig = px.area(long_df, x='Year', y='Amount', color='Grouped Sector')
    fig.update_layout(
        paper_bgcolor='white',
        plot_bgcolor='white',
        )
    
    return fig



In [312]:
showfig = stacked_area_chart(sector_anal_df, 2014, 2021, "Gesamt ODA", 5, "Anteilig")
showfig

,Grouped Sector,Year,Amount
0,Andere Sektoren,2013,3.125743e+09
1,I.5.a. Government & Civil Society-general,2013,7.913002e+08
2,I.6. Other Social Infrastructure & Services,2013,8.745827e+07
3,III.1.a. Agriculture,2013,2.176680e+08
4,IV.1. General Environment Protection,2013,3.058654e+08
5,IV.2. Other Multisector,2013,5.877497e+08
6,Andere Sektoren,2014,4.053460e+09
7,I.5.a. Government & Civil Society-general,2014,8.870652e+08
8,I.6. Other Social Infrastructure & Services,2014,9.371650e+07
9,III.1.a. Agriculture,2014,2.866558e+08


In [295]:
showfig.columns

Index(['amount_2013', 'amount_2014', 'amount_2015', 'amount_2016',
       'amount_2017', 'amount_2018', 'amount_2019', 'amount_2020',
       'amount_2021', 'amount_2022', 'Sector'],
      dtype='object')

In [301]:
showfig.melt(id_vars=['Sector'], var_name='Year', value_name='Amount')[['Amount', 'Sector']].groupby('Sector').mean()


,Amount
Sector,
"I.1.a. Education, Level Unspecified",1.453964e+08
I.1.b. Basic Education,2.331104e+08
I.1.c. Secondary Education,2.315269e+08
I.1.d. Post-Secondary Education,9.926801e+07
"I.2.a. Health, General",1.212209e+08
I.2.b. Basic Health,3.699082e+08
I.2.c. Non-communicable diseases (NCDs),1.809513e+07
I.3. Population Policies/Programmes & Reproductive Health,1.635230e+08
I.4. Water Supply & Sanitation,4.237881e+08


In [284]:
showfig = stacked_area_chart(df_merged, 2014, 2020, "Klimaschutz", 10, "abs")
showfig.show()

In [274]:
total_per_year = longest_df.groupby('Year')['Amount'].sum().reset_index(name='Total')
longest_df = pd.merge(longest_df, total_per_year, on='Year')


In [275]:
longest_df['Percentage'] = (longest_df['Amount'] / longest_df['Total']) * 100


In [276]:
# Step 4: Create the plot using the 'Percentage' as the y-value and stack the data
fig = px.area(longest_df, x='Year', y='Percentage', color='Grouped Sector',
              labels={'Percentage': 'Percentage of Total'},
              title='Stacked Area Plot of Grouped Sector as Percentage of Total per Year')

# Update layout for a cleaner background as you preferred before
fig.update_layout(
    paper_bgcolor='white',
    plot_bgcolor='white',
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False, ticksuffix="%")  # Add a percentage sign to y-axis ticks
)

# Show the figure
fig.show()

In [254]:
showfig = stacked_area_chart(df_merged, 2014, 2021, "Klimaschutz", 6, "abs")
showfig.show()

In [201]:
df_long = df.melt(id_vars=['Grouped Sector'], var_name='Year', value_name='Amount')


In [202]:
df_long

,Grouped Sector,Year,Amount
0,Other,amount_2013,87245749.0
1,Other,amount_2013,153930790.0
2,Other,amount_2013,110570602.0
3,Other,amount_2013,104939416.0
4,Other,amount_2013,95765407.0
...,...,...,...
413,Other,Sector,VIII.1. Emergency Response
414,Other,Sector,VIII.2. Reconstruction Relief & Rehabilitation
415,Other,Sector,VIII.3. Disaster Prevention & Preparedness
416,Other,Sector,II.3.d. Hybrid energy plants


In [191]:


# Melt the adjusted DataFrame
df_long = df.melt(id_vars=['Grouped Sector'], var_name='Year', value_name='Amount')

# Sum amounts for 'Other' by year
df_long = df_long.groupby(['Grouped Sector', 'Year']).sum().reset_index()

# Extract year from the 'Year' column
df_long['Year'] = df_long['Year'].str.extract('(\d+)')

# Convert the Year column to numeric
df_long['Year'] = pd.to_numeric(df_long['Year'])

# Create a filled area plot
fig = px.area(df_long, x='Year', y='Amount', color='Grouped Sector', line_group='Grouped Sector')

# Show the plot
fig.show()

In [187]:
# Create a filled area plot
fig = px.area(df_long, x='Year', y='Amount', color='Sector', line_group='Sector')

# Show the plot
fig.show()

## Pie Charts

In [102]:
list(range(2013,2023))

[2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

In [103]:
year = 2013

selected_year = [col for col in df_merged.columns if col.endswith(f'{year}')]
selected_year.append('Sector')
filtered_year_df = df_merged[selected_year]

In [110]:
clim_rel_fig = px.sunburst(filtered_year_df, path=['Sector'], values=f'clim_rel_amount_{year}', title='Climate Related Amount by Sector')

layout_update = {
    'margin': dict(t=30, l=0, r=0, b=0)  # Adjust top, left, right, bottom margins as needed
}

clim_rel_fig.update_layout(**layout_update)


clim_rel_fig.show()


In [213]:
non_clim_fig = px.sunburst(filtered_year_df, path=['Sector'], values=f'non_clim_{year}', title='Non Climate Related Amount by Sector')
non_clim_fig.show()


In [214]:
clim_adapt_fig = px.sunburst(filtered_year_df, path=['Sector'], values=f'clim_adapt_amount_{year}', title='Climate Apaptation Amount by Sector')
clim_adapt_fig.show()


In [215]:
clim_miti_fig = px.sunburst(filtered_year_df, path=['Sector'], values=f'clim_miti_amount_{year}', title='Climate Mitigation Amount by Sector')
clim_miti_fig.show()


## By Country

In [235]:
use_df

,Recipient Name,Amount,Climate Mitigation,Climate Adaptation,Sector,clim_rel,clim_rel_amount,clim_adapt_amount,clim_miti_amount
276,Egypt,0.807941,0.0,0.0,IV.1. General Environment Protection,0.0,0.000000,0.0,0.000000
277,Egypt,0.041383,0.0,0.0,IV.1. General Environment Protection,0.0,0.000000,0.0,0.000000
278,Egypt,2.600523,1.0,0.0,IV.1. General Environment Protection,0.5,1.300261,0.0,1.300261
280,Egypt,0.217925,0.0,0.0,IV.2. Other Multisector,0.0,0.000000,0.0,0.000000
317,Egypt,0.021100,0.0,0.0,IV.2. Other Multisector,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...
12164,Syrian Arab Republic,0.011091,0.0,0.0,IV.2. Other Multisector,0.0,0.000000,0.0,0.000000
12165,West Bank and Gaza Strip,0.055108,0.0,0.0,IV.2. Other Multisector,0.0,0.000000,0.0,0.000000
12173,Yemen,0.010539,0.0,0.0,IV.2. Other Multisector,0.0,0.000000,0.0,0.000000
12174,"Middle East, regional",0.029859,0.0,0.0,IV.2. Other Multisector,0.0,0.000000,0.0,0.000000


In [249]:
def determine_value(row):
    if 2.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 1
    elif row['Climate Adaptation'] == 1.0 and row['Climate Mitigation'] == 1.0:
        return 1
    elif 1.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 0.5
    else:
        return 0

# final_dict = dictionary_df.copy()    
processed_grouped_dfs = []

    
for year, df in dictionary_df.items():
    
    # Data Cleaning
    use_df = df[['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation', 'Sector']]
    use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
    use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)
    
    # Applying analysis Rules
    
    use_df['clim_rel'] = use_df.apply(determine_value, axis=1)
    
    use_df['clim_rel_amount'] = use_df['Amount'] * use_df['clim_rel']
    use_df['clim_adapt_amount'] = use_df['Amount'] * (use_df['Climate Adaptation'] / 2)
    use_df['clim_miti_amount'] = use_df['Amount'] * (use_df['Climate Mitigation'] / 2)
    
    
    # Grouping by Country and calculating
    
    grouped_df = use_df.groupby(['Sector', 'Recipient Name']).sum()[['Amount','clim_rel_amount', 'clim_adapt_amount', 'clim_miti_amount']]
    
    grouped_df['non_miti_adapt'] =grouped_df['Amount'] - (grouped_df['clim_miti_amount'] + grouped_df['clim_adapt_amount'])
    grouped_df['non_clim'] =grouped_df['Amount'] - grouped_df['clim_rel_amount']
    
        
    grouped_df = grouped_df.rename(columns= {
        'Amount': f"amount_{year}",
        'non_miti_adapt': f"non_miti_adapt_{year}",
        'clim_rel_amount': f"clim_rel_amount_{year}",
        'non_clim': f"non_clim_{year}",
        'clim_adapt_amount': f"clim_adapt_amount_{year}",
        'clim_miti_amount': f"clim_miti_amount_{year}"
    })
    
    # Merging DF
    
    processed_grouped_dfs.append(grouped_df)
    



In [251]:
processed_grouped_dfs[0]

amount_2013  \
Sector                                     Recipient Name                          
I.1.a. Education, Level Unspecified        Afghanistan                  0.020691   
                                           Africa, regional             2.571572   
                                           America, regional            2.482967   
                                           Asia, regional               4.524518   
                                           Belarus                      0.015519   
...                                                                          ...   
VIII.3. Disaster Prevention & Preparedness Pakistan                     0.758688   
                                           Philippines                  0.187683   
                                           Somalia                      0.758685   
                                           West Bank and Gaza Strip     0.386239   
                                           Yemen                        0.178599   

                                                                     clim_rel_amount_2013  \
Sector                                     Recipient Name                                   
I.1.a. Education, Level Unspecified        Afghanistan                           0.000000   
                                           Africa, regional                      0.000000   
                                           America, regional                     0.000000   
                                           Asia, regional                        2.262259   
                                           Belarus                               0.000000   
...                                                                                   ...   
VIII.3. Disaster Prevention & Preparedness Pakistan                              0.758688   
                                           Philippines                           0.090172   
                                           Somalia                               0.758685   
                                           West Bank and Gaza Strip              0.193119   
                                           Yemen                                 0.000000   

                                                                     clim_adapt_amount_2013  \
Sector                                     Recipient Name                                     
I.1.a. Education, Level Unspecified        Afghanistan                             0.000000   
                                           Africa, regional                        0.000000   
                                           America, regional                       0.000000   
                                           Asia, regional                          2.262259   
                                           Belarus                                 0.000000   
...                                                                                     ...   
VIII.3. Disaster Prevention & Preparedness Pakistan                                0.379344   
                                           Philippines                             0.090172   
                                           Somalia                                 0.758685   
                                           West Bank and Gaza Strip                0.193119   
                                           Yemen                                   0.000000   

                                                                     clim_miti_amount_2013  \
Sector                                     Recipient Name                                    
I.1.a. Education, Level Unspecified        Afghanistan                            0.000000   
                                           Africa, regional                       0.000000   
                                           America, regional                      0.000000   
                                           Asia, regional   

In [252]:
df_merged = reduce(lambda left, right: pd.merge(left, right, on=['Sector', 'Recipient Name'], how='outer'), processed_grouped_dfs)


In [255]:
df_merged.to_csv('../upload_data/country_sector_analysis.csv')

df_merged = pd.read_csv('../upload_data/country_sector_analysis.csv')

In [257]:
year = 2013
country = 'Afghanistan'

selected_year = [col for col in df_merged.columns if col.endswith(f'{year}')]
selected_year.append('Sector')
df_merged = df_merged[df_merged['Recipient Name'] == country]
filtered_year_df = df_merged[selected_year]

In [258]:
filtered_year_df

,amount_2013,clim_rel_amount_2013,clim_adapt_amount_2013,clim_miti_amount_2013,non_miti_adapt_2013,non_clim_2013,Sector
0,0.020691,0.000000,0.000000,0.000000,0.020691,0.020691,"I.1.a. Education, Level Unspecified"
48,35.795844,0.000000,0.000000,0.000000,35.795844,35.795844,I.1.b. Basic Education
116,22.151253,1.888354,0.944177,0.944177,20.262899,20.262899,I.1.c. Secondary Education
221,1.293674,0.000000,0.000000,0.000000,1.293674,1.293674,"I.2.a. Health, General"
267,0.904807,0.000000,0.000000,0.000000,0.904807,0.904807,I.2.b. Basic Health
322,0.693010,0.000000,0.000000,0.000000,0.693010,0.693010,I.3. Population Policies/Programmes & Reproduc...
383,22.726831,3.212090,3.212090,0.000000,19.514740,19.514740,I.4. Water Supply & Sanitation
456,54.185554,0.000000,0.000000,0.000000,54.185554,54.185554,I.5.a. Government & Civil Society-general
562,13.003176,0.000000,0.000000,0.000000,13.003176,13.003176,"I.5.b. Conflict, Peace & Security"
687,8.093760,0.000000,0.000000,0.000000,8.093760,8.093760,II.1. Transport & Storage


In [9]:
dictionary_df['2015'][['Sector', 'SECTOR']]

,Sector,SECTOR
0,IV.1. General Environment Protection,41010
1,IV.1. General Environment Protection,41020
2,IV.2. Other Multisector,43010
3,IV.2. Other Multisector,43010
4,IV.2. Other Multisector,43030
...,...,...
12596,I.1.b. Basic Education,11230
12598,IV.2. Other Multisector,43010
12601,IV.2. Other Multisector,43010
12611,I.5.a. Government & Civil Society-general,15110
